In [1]:
import real_robot.envs
import gym

env = gym.make("PlaceCubeInBowlRealXArm-v8", control_mode="pd_ee_pose",
               obs_mode="rgbd", reward_mode="grounded_sam_sparse_staged_v3",
               image_obs_mode="front", robot_action_scale=100)
env._max_episode_steps = 400
obs = env.reset()

SDK_VERSION: 1.11.6


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])

Loading GroundingDINO: Took 4.034 seconds


In [2]:
env.use_grounded_sam, env.recent_sam_obs

(True,
 OrderedDict([('sam_rgb_images', array([[[[ 11,  21,  24],
                        [ 10,  20,  23],
                        [ 11,  21,  26],
                        ...,
                        [ 28,  36,  33],
                        [ 29,  36,  28],
                        [ 27,  34,  26]],
               
                       [[ 10,  20,  34],
                        [  8,  18,  32],
                        [ 11,  22,  33],
                        ...,
                        [ 29,  39,  25],
                        [ 27,  40,  23],
                        [ 26,  39,  22]],
               
                       [[  7,  15,  41],
                        [  4,  12,  38],
                        [  2,  14,  30],
                        ...,
                        [ 22,  40,  17],
                        [ 25,  47,  18],
                        [ 22,  44,  15]],
               
                       ...,
               
                       [[ 76,  83,  87],
              

In [14]:
env.recent_sam_obs.keys(), env.recent_sam_obs["object_pcds"]

(odict_keys(['sam_rgb_images', 'sam_xyz_images', 'sam_xyz_masks', 'pred_masks', 'pred_phrases', 'boxes_filt', 'object_pcds', 'object_filt_pcds']),
 {'red cube': array([[ 0.27719426, -1.06621003,  0.35899004],
         [ 0.27719426, -1.06621003,  0.35899004],
         [ 0.27719426, -1.06621003,  0.35899004],
         ...,
         [ 0.48136342, -0.06042967,  0.3297182 ],
         [ 0.48136342, -0.06042967,  0.3297182 ],
         [ 0.48136342, -0.06042967,  0.3297182 ]]),
  'green bowl': array([[ 0.27719426, -1.06621003,  0.35899004],
         [ 0.27719426, -1.06621003,  0.35899004],
         [ 0.27719426, -1.06621003,  0.35899004],
         ...,
         [ 0.48136342, -0.06042967,  0.3297182 ],
         [ 0.48136342, -0.06042967,  0.3297182 ],
         [ 0.48136342, -0.06042967,  0.3297182 ]])})

In [9]:
from collections import defaultdict
self = env
obs_dict = defaultdict(list)
for cam_name, img_dict in self.recent_camera_obs.items():
    obs_dict["camera_names"].append(cam_name)
    obs_dict["color_images"].append(img_dict["rgb"])
    if "depth" in img_dict:
        obs_dict["depth_images"].append(img_dict["depth"])

if getattr(self, "recent_sam_obs", None) is not None:
    obs_dict["color_images"] = list(self.recent_sam_obs["sam_rgb_images"])
    obs_dict.pop("depth_images")  # FIXME: visualize resized depth images
    obs_dict["pred_masks"] = list(self.recent_sam_obs["pred_masks"])
    obs_dict["xyz_images"] = list(self.recent_sam_obs["sam_xyz_images"])
    obs_dict["cube_pts"] = self.recent_sam_obs["object_pcds"]["red cube"]
    obs_dict["bowl_pts"] = self.recent_sam_obs["object_pcds"]["green bowl"]
    obs_dict["cube_filt_pts"] = self.recent_sam_obs["object_filt_pcds"]["red cube"]
    obs_dict["bowl_filt_pts"] = self.recent_sam_obs["object_filt_pcds"]["green bowl"]
self.visualizer.show_observation(**obs_dict)
self.visualizer.render()

KeyboardInterrupt: 

In [2]:
import numpy as np
self = env

cube_pts = self.recent_sam_obs["object_filt_pcds"]["red cube"]
bowl_pts = self.recent_sam_obs["object_filt_pcds"]["green bowl"]
# Extract bbox from object_pts
cube_mins, cube_maxs = cube_pts.min(0), cube_pts.max(0)
bowl_mins, bowl_maxs = bowl_pts.min(0), bowl_pts.max(0)
# Extract position from bbox
cube_pos = np.mean([cube_mins, cube_maxs], axis=0)
bowl_pos = np.mean([bowl_mins, bowl_maxs], axis=0)
cube_pos, bowl_pos

(array([ 0.20701366, -0.36612499,  0.01739574]),
 array([ 0.29024439, -0.19213866,  0.02106138]))

In [6]:
above_cube_pose = env.agent.build_grasp_pose(cube_pos + [0, 0, 0.1])
cube_pose = env.agent.build_grasp_pose(cube_pos)
above_bowl_pose = env.agent.build_grasp_pose(bowl_pos + [0, 0, 0.1])
bowl_pose = env.agent.build_grasp_pose(bowl_pos + [0, 0, 0.01])

above_cube_pose, cube_pose, above_bowl_pose, bowl_pose

(Pose([0.207014, -0.366125, 0.117396], [0, 0.707107, 0.707107, 0]),
 Pose([0.207014, -0.366125, 0.0173957], [0, 0.707107, 0.707107, 0]),
 Pose([0.290244, -0.192139, 0.121061], [0, 0.707107, 0.707107, 0]),
 Pose([0.290244, -0.192139, 0.0310614], [0, 0.707107, 0.707107, 0]))

In [7]:
from transforms3d.euler import quat2euler
def get_action(pose, close_gripper=False):
    action = np.hstack([pose.p, quat2euler(pose.q), [-1 if close_gripper else 1]])
    return action

env.step_action(get_action(above_cube_pose))
env.step_action(get_action(cube_pose))
env.step_action(get_action(cube_pose, True))
env.step_action(get_action(above_cube_pose, True))
env.step_action(get_action(above_bowl_pose, True))
env.step_action(get_action(bowl_pose, True))
env.step_action(get_action(bowl_pose, False))

XArm7 - (xarm.py:171) - INFO - 2023-07-26 20:51 - Setting action = array([-3.66122772e+02, -2.56851318e+02,  6.94740753e+01,  5.87305612e-06,
        4.98176093e-06,  2.92064222e-06,  8.50000000e+02])


tgt_tcp_pose = Pose([207.014, -366.125, 117.396], [4.32978e-17, 0.707107, 0.707107, 4.32978e-17])


XArm7 - (xarm.py:171) - INFO - 2023-07-26 20:51 - Setting action = array([-4.57763672e-04, -3.20434570e-04,  1.00000145e+02, -1.50995789e-07,
        2.32251577e-08, -5.96046465e-08,  8.50000000e+02])


tgt_tcp_pose = Pose([207.014, -366.125, 17.3957], [4.32978e-17, 0.707107, 0.707107, 4.32978e-17])


XArm7 - (xarm.py:171) - INFO - 2023-07-26 20:51 - Setting action = array([ 4.88281250e-04,  3.35693359e-04,  5.24520874e-04, -1.50995790e-07,
        4.32659588e-08, -5.96046480e-08, -1.00000000e+01])


...

tgt_tcp_pose = Pose([290.244, -192.139, 121.061], [4.32978e-17, 0.707107, 0.707107, 4.32978e-17])


XArm7 - (xarm.py:171) - INFO - 2023-07-26 20:52 - Setting action = array([ 5.79833984e-04,  3.35693359e-04,  9.00005951e+01, -8.74228051e-08,
       -1.52307397e-07,  1.19209303e-07, -1.00000000e+01])


tgt_tcp_pose = Pose([290.244, -192.139, 31.0614], [4.32978e-17, 0.707107, 0.707107, 4.32978e-17])


XArm7 - (xarm.py:171) - INFO - 2023-07-26 20:52 - Setting action = array([-5.34057617e-04, -2.74658203e-04,  6.14166260e-04, -1.50995815e-07,
       -5.39777374e-08,  1.19209301e-07,  8.50000000e+02])


tgt_tcp_pose = Pose([290.244, -192.139, 31.0614], [4.32978e-17, 0.707107, 0.707107, 4.32978e-17])


In [8]:
env.agent.reset()

************* GetErrorWarnCode, Status: 0 **************
* ErrorCode: 0, Info: Normal
* WarnCode: 0, Info: Normal
**************************************************
[motion_enable], xArm is not ready to move
[set_state], xArm is ready to move
